Trabalho Prático 1 - Geometria computacional

1. Implementar um algoritmo de envoltória convexa visto em sala de aula

Deborah: sugiro o Gift Wrapping por ter complexidade O(nh) e não necessitar de ordenação. Seria interessante usar o algoritmo incremental v1 para adicionar novos pontos, mas de qualquer forma cada adição é O(n²).
https://www.cs.jhu.edu/~misha/Spring16/07.pdf


In [ ]:
from envoltoria import envoltoria_convexa

2. Implementar o algoritmo de varredura linear para detecção de interseções em conjuntos de segmentos

3. Implementar o método para verificação de separabilidade linear conforme descrito acima

Para verificar a separabilidade dos dados, o método deverá computar as envoltórias convexas de cada classe (será assumido que os dados possuem apenas dois rótulos para simplificar). Em seguida, deverá ser verificado se há ou não sobreposição dessas classes/envoltórias através do algoritmo de varredura linear para verificação de interseção de segmentos. Caso não haja interseção entre segmentos de envoltórias
distintas, então os dados são linearmente separáveis.

4. Implementar o método para construir o modelo, caso os dados sejam linearmente separáveis

In [ ]:
from modelo import *

5. Implementar o classificador que recebe um conjunto de amostras desconhecidas e atribui rótulos a elas

6. Implementar o método para computar as métricas de classificação para os experimentos

7. Realizar os experimentos conforme a descrição acima
8. Redigir o relatório especificado acima